In [1]:
import torch
import torch.nn.functional as F
torch.set_printoptions(profile='full')

/home/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
a = torch.Tensor([[1,2,3], [4,5,6]])
b = torch.Tensor([[7,8,9], [10,11,12], [13,14,15]])
torch.cat((a,b),dim=0)

tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.],
        [13., 14., 15.]])

In [14]:
embedding = torch.randn(5)
F.normalize(embedding, dim=0)

tensor([-0.5405, -0.0218, -0.3455,  0.1786,  0.7457])

In [15]:
batch = F.normalize(torch.randn(10,5), dim=1)
batch

tensor([[-0.1202,  0.0534,  0.4551, -0.8527, -0.2203],
        [-0.3674,  0.8867,  0.2422, -0.1004, -0.1002],
        [-0.6232,  0.7248,  0.2256,  0.1132,  0.1500],
        [ 0.4695,  0.0916, -0.2668, -0.4781, -0.6866],
        [-0.4618,  0.0426,  0.6552, -0.5936,  0.0577],
        [-0.5506,  0.2652,  0.3107, -0.2665,  0.6775],
        [ 0.5242,  0.3289,  0.3215, -0.4149, -0.5844],
        [-0.1140,  0.3257,  0.7817,  0.3853, -0.3486],
        [ 0.6502, -0.3069, -0.0812,  0.0064, -0.6902],
        [-0.2065, -0.6311,  0.5693,  0.0159,  0.4845]])

In [16]:
labels = torch.Tensor([0,1,1,0,0,2,2,0,3,3])

In [17]:
temperature = 0.01

In [42]:
dot_product_tempered = torch.mm(batch, batch.T) / temperature  # z_i \dot z_j / \tau
dot_product_tempered.size()  # [dim, dim](10,10)
dot_product_tempered

tensor([[100.0000,  30.9469,   8.6748,  38.5924,  84.9417,  29.9760,  58.3367,
          13.5135,   1.5001,  12.9935],
        [ 30.9469, 100.0000,  89.9913,  -3.9088,  41.9938,  47.1563,  27.7121,
          51.6273, -46.2144, -39.5910],
        [  8.6748,  89.9913, 100.0000, -44.3535,  40.7908,  67.6953, -15.0436,
          47.4779, -74.8818, -12.5751],
        [ 38.5924,  -3.9088, -44.3535, 100.0000, -14.3502, -65.4863,  79.0061,
         -17.7123,  76.9650, -64.6936],
        [ 84.9417,  41.9938,  40.7908, -14.3502, 100.0000,  66.6372,  19.5193,
          32.9882, -41.0162,  46.0017],
        [ 29.9760,  47.1563,  67.6953, -65.4863,  66.6372, 100.0000, -38.6880,
           5.3235, -93.3951,  44.7266],
        [ 58.3367,  27.7121, -15.0436,  79.0061,  19.5193, -38.6880, 100.0000,
          34.2547,  61.4509, -42.2503],
        [ 13.5135,  51.6273,  47.4779, -17.7123,  32.9882,   5.3235,  34.2547,
         100.0000,   0.5503,  10.0281],
        [  1.5001, -46.2144, -74.8818,  76.9650,

In [43]:
max_ele = torch.max(dot_product_tempered, dim=1, keepdim=True)
print(max_ele)
dot_product_tempered = dot_product_tempered - max_ele[0]
print(dot_product_tempered)
exp_dot_tempered = (
    torch.exp(dot_product_tempered) 
    + 1e-5
)  # exp(z_i \dot z_j / \tau)，每一行代表和每个其他嵌入的点积
exp_dot_tempered

torch.return_types.max(
values=tensor([[100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000],
        [100.0000]]),
indices=tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]]))
tensor([[   0.0000,  -69.0531,  -91.3252,  -61.4076,  -15.0583,  -70.0240,
          -41.6633,  -86.4865,  -98.4999,  -87.0065],
        [ -69.0531,    0.0000,  -10.0088, -103.9088,  -58.0062,  -52.8438,
          -72.2879,  -48.3727, -146.2144, -139.5911],
        [ -91.3252,  -10.0088,    0.0000, -144.3535,  -59.2092,  -32.3047,
         -115.0436,  -52.5221, -174.8818, -112.5751],
        [ -61.4076, -103.9088, -144.3535,    0.0000, -114.3502, -165.4863,
          -20.9939, -117.7123,  -23.0350, -164.6936],
        [ -15.0583,  -58.0062,  -59.2092, -114.3502,    0.0000,  -33.3628,
          -80.4807,  -67.0118, -141.

tensor([[1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0289e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e+00, 5.5004e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 5.5004e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05, 1.0000e-05,
         1.0001e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0289e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00,
         1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0001e-05, 1.0000e-05, 1.0000e-05,
         1.0000e+00, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000

In [44]:
# 分母是z_i与batch中其他嵌入的点积，因此要屏蔽掉exp_dot_tempered中自己与自己相乘的部分，即对角线上的取值
unsqeezed_labels = labels.unsqueeze(1)  # [[0],[1],[1],[0],[0],[2],[2],[0],[3],[3]]
repeated_labels = unsqeezed_labels.repeat(1, labels.shape[0])  # 每一行都是batch_size长的全为对应的样本的标签的向量
mask_similar_class = (repeated_labels == labels)
print(mask_similar_class)
mask_anchor_out = (1 - torch.eye(exp_dot_tempered.shape[0]))  # 对角线为0其他全为1的batch_size * batch_size的张量
print(mask_anchor_out)

tensor([[ True, False, False,  True,  True, False, False,  True, False, False],
        [False,  True,  True, False, False, False, False, False, False, False],
        [False,  True,  True, False, False, False, False, False, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [False, False, False, False, False,  True,  True, False, False, False],
        [False, False, False, False, False,  True,  True, False, False, False],
        [ True, False, False,  True,  True, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True]])
tensor([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],

In [21]:
mask_combined = mask_similar_class * mask_anchor_out
mask_combined  # 每个样本在batch中相同样本的其他样本对应的位置为1，不同的为0

tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [45]:
cardinality_per_samples = torch.sum(mask_combined, dim=1)
cardinality_per_samples  # batch中每个样本的与他标签相同的其他样本的数量

tensor([3., 1., 1., 3., 3., 1., 1., 3., 1., 1.])

In [46]:
fenmu = torch.sum(exp_dot_tempered * mask_anchor_out, dim=1, keepdim=True)
print(fenmu)
log_prob = -torch.log(exp_dot_tempered / fenmu)
print(log_prob)

tensor([[9.0289e-05],
        [1.3500e-04],
        [1.3500e-04],
        [9.0001e-05],
        [9.0289e-05],
        [9.0000e-05],
        [9.0001e-05],
        [9.0000e-05],
        [9.0000e-05],
        [9.0000e-05]])
tensor([[-9.3125,  2.2004,  2.2004,  2.2004,  2.1720,  2.2004,  2.2004,  2.2004,
          2.2004,  2.2004],
        [ 2.6027, -8.9102,  0.8979,  2.6027,  2.6027,  2.6027,  2.6027,  2.6027,
          2.6027,  2.6027],
        [ 2.6027,  0.8979, -8.9102,  2.6027,  2.6027,  2.6027,  2.6027,  2.6027,
          2.6027,  2.6027],
        [ 2.1972,  2.1972,  2.1972, -9.3157,  2.1972,  2.1972,  2.1972,  2.1972,
          2.1972,  2.1972],
        [ 2.1720,  2.2004,  2.2004,  2.2004, -9.3125,  2.2004,  2.2004,  2.2004,
          2.2004,  2.2004],
        [ 2.1972,  2.1972,  2.1972,  2.1972,  2.1972, -9.3157,  2.1972,  2.1972,
          2.1972,  2.1972],
        [ 2.1972,  2.1972,  2.1972,  2.1972,  2.1972,  2.1972, -9.3157,  2.1972,
          2.1972,  2.1972],
        [ 2.1972

In [47]:
log = log_prob * mask_combined
print(log)
supervised_contrastive_loss_per_sample = torch.sum(log, dim=1) / cardinality_per_samples
print(supervised_contrastive_loss_per_sample)
supervised_contrastive_loss = torch.sum(supervised_contrastive_loss_per_sample)
print(supervised_contrastive_loss)


tensor([[-0.0000, 0.0000, 0.0000, 2.2004, 2.1720, 0.0000, 0.0000, 2.2004, 0.0000,
         0.0000],
        [0.0000, -0.0000, 0.8979, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.8979, -0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [2.1972, 0.0000, 0.0000, -0.0000, 2.1972, 0.0000, 0.0000, 2.1972, 0.0000,
         0.0000],
        [2.1720, 0.0000, 0.0000, 2.2004, -0.0000, 0.0000, 0.0000, 2.2004, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0000, 2.1972, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972, -0.0000, 0.0000, 0.0000,
         0.0000],
        [2.1972, 0.0000, 0.0000, 2.1972, 2.1972, 0.0000, 0.0000, -0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0000,
         2.1972],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1972,
         -0.0000]])